In [2]:
!pip install triton

In [3]:
import torch
import triton
import triton.language as tl

In [4]:
@triton.jit
def multipleby2(inputptr,outputptr,n,block_size: tl.constexpr):
    blockid=tl.program_id(0)
    block_start= blockid * block_size 
    offsets=block_start+tl.arange(0,block_size)
    mask=offsets<n
    x=tl.load(inputptr+offsets,mask=mask)
    result=x*2
    tl.store(outputptr+offsets,result,mask=mask)

In [6]:
def test():
    data = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], device='cuda', dtype=torch.float32)
    n=len(data)

    output2 = torch.zeros_like(data)
    BLOCK_SIZE = 4  
    num_blocks = triton.cdiv(n, BLOCK_SIZE)  
    multipleby2[(num_blocks,)](data, output2, n, BLOCK_SIZE)
    print(f"New way result: {output2}")

if __name__ == "__main__":
    test()

New way result: tensor([ 2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.], device='cuda:0')
